# Running inference

In [2]:
import librosa as lr
import octafilt3r.filter as o3f
import os
from tensorflow.keras.models import load_model
import numpy as np
import pipeline.config as conf
import pipeline.scale as scale

In [6]:
param_dict = conf.open_params()

base_path = "Datasets/UrbanSound8k_augmented/street_music/"
stop_at = 30
i = 0
true = 0
false = 0

N4CED = load_model(param_dict["CNN_name"])

for file in os.listdir(base_path):

    if(i == stop_at):
        break

    """
    LOAD DATA
    """
    wav, fs = lr.load(base_path + file, sr=param_dict['fs'], duration=param_dict["max_dur"])

    if len(wav) < fs:
        continue


    """
    EXTRACT FEATURES
    """
    feats, fcs = o3f.rolling_oct_bank(
        wav,
        fs,
        param_dict['oct_bw_ratio'],
        param_dict['order'],
        param_dict['fmax'],
        param_dict['fmin'],
        param_dict['frame_size'],
        param_dict['dec_stages'],
        param_dict['dec_ord']
    )


    """
    RESHAPE FEATURES
    """
    df = scale.feats2frames(feats, param_dict['frame_size'], fs)


    """
    SCALE FEATURES
    """
    X, scaler = scale.data_scaler(df)


    """
    RUN MODEL
    """
    p = N4CED.predict(X)


    """
    PROCESS RESULTS
    """
    predicted = param_dict["class_map"][np.argmax(np.mean(p, axis=0))]
    print(f'Predicted class: "{predicted}" with {int(100 * (p[0][np.argmax(p, 1)][0]))} score.')
    if predicted == 'street_music':
        true += 1
    else:
        false += 1
    i += 1

print('----- done. -----')
print(f'{true} true predictions\n{false} false predictions')
print(f'test accuracy: {int((true/i)*100)}%\nerror ratio: {int((false/i)*100)}%')

Predicted class: "street_music" with 93 score.
Predicted class: "street_music" with 73 score.
Predicted class: "street_music" with 90 score.
Predicted class: "street_music" with 93 score.
Predicted class: "street_music" with 92 score.
Predicted class: "street_music" with 77 score.
Predicted class: "dog_bark" with 59 score.
Predicted class: "street_music" with 38 score.
Predicted class: "children_playing" with 55 score.
Predicted class: "children_playing" with 53 score.
Predicted class: "street_music" with 41 score.
Predicted class: "street_music" with 43 score.
Predicted class: "street_music" with 95 score.
Predicted class: "street_music" with 74 score.
Predicted class: "street_music" with 49 score.
Predicted class: "street_music" with 29 score.
Predicted class: "drilling" with 51 score.
Predicted class: "street_music" with 67 score.
Predicted class: "street_music" with 97 score.
Predicted class: "street_music" with 90 score.
Predicted class: "street_music" with 64 score.
Predicted cla

In [7]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(N4CED)
tflite_model = converter.convert()
open(param_dict["CNN_name"] + "/" + param_dict["CNN_name"] + "_lite.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\JDA1E~1.TSC\AppData\Local\Temp\tmp5his_7ln\assets


745868